In [1]:
import numpy as np
from scipy.linalg import solve_triangular
import time

## Decomposição LU

In [2]:
def lu_decomposition(A, b):
    U = A.copy()
    b = b.copy()
    
    n = len(b)
    L = np.eye(n)
    
    for i in range(n):
        pivot = U[i, i]
            
        for k in range(i+1, n):
            m = U[k, i]/pivot
            U[k] = U[k] - U[i]*m
            b[k] = b[k] - b[i]*m
            
            L[k, i] = m
    
    return L, U, b

In [3]:
A = np.array([[-1,2,-2,1], [-2,2,-3,0], [-2,0,-1,-3], [-2,0,0,-2]])
b = np.array([1,1,1,1])

L, U, b = lu_decomposition(A, b)

In [4]:
A = np.array([[1, 1], 
              [0, 0]])

b = np.array([1,1])

L, U, b = lu_decomposition(A, b)
L@U

array([[1., 1.],
       [0., 0.]])

In [5]:
E1 = np.array([[1,0,0,0], 
               [2,1,0,0], 
               [2,0,1,0], 
               [2,0,0,1]])

E2 = np.array([[1,0,0,0], 
               [0,1,0,0], 
               [0,2,1,0], 
               [0,2,0,1]])

E3 = np.array([[1,0,0,0], 
               [0,1,0,0], 
               [0,0,1,0], 
               [0,0,2,1]])

E1@E2@E3

array([[1, 0, 0, 0],
       [2, 1, 0, 0],
       [2, 2, 1, 0],
       [2, 2, 2, 1]])

In [6]:
U

array([[1, 1],
       [0, 0]])

## Eliminação Gaussiana

In [7]:
def gaussian_elimination(A, b):
    A = A.copy()
    b = b.copy()
    
    n = len(b)
    
    for i in range(n):
        pivot = A[i, i]
            
        for k in range(i+1, n):
            m = A[k, i]/pivot
            A[k] = A[k] - A[i]*m
            b[k] = b[k] - b[i]*m
    
    return A, b

In [8]:
def solve_upper(A, b):
    A = A.copy()
    b = b.copy()
    
    n = len(b)
    solution = []
    
    for i in range(n-1, -1, -1):
        sum_ = np.sum(A[i, i+1:n])
        current_solution = (b[i] - sum_)/A[i,i]
    
        solution.append(current_solution)
        A[:, i] = A[:, i] * current_solution
        
    return solution[::-1]

In [9]:
def solve_lower(A, b):
    A = A.copy()
    b = b.copy()
    
    n = len(b)
    solution = []
    
    for i in range(n):
        sum_ = np.sum(A[i, 0:i])
        current_solution = (b[i] - sum_)/A[i,i]
    
        solution.append(current_solution)
        A[:, i] = A[:, i] * current_solution
        
    return solution

## Decomposição de Cholesky

<img src="images/cholesky.jpg" width="500">

In [10]:
def cholesky(A):
    n = len(A)
    L = np.zeros((n, n))
    
    L[0, 0] = np.sqrt(A[0, 0])
    
    for i in range(1, n):
        for j in range(0, i+1):
            sum_ = 0
            for k in range(0, j):
                sum_ += L[i, k] * L[j, k]
            
            if i == j:           
                L[i, i] = np.sqrt(A[i, i] - sum_)
            else:
                L[i, j] = (A[i, j] - sum_)/L[j, j]
    
    return L, L.transpose()

In [11]:
def solve_cholesky(A, b):
    # https://homepages.dcc.ufmg.br/~ana.coutosilva/Aulas-CN/SistemasLineares/Cholesky.pdf
    
    L, Lt = cholesky(A)
    y = solve_lower(L, b)
    
    solution = solve_upper(Lt, y)
    return solution

In [12]:
import matplotlib.pyplot as plt

## Gauss-Jacobi

![jacobi](images/jacobi.png)

In [13]:
def jacobi(A, b):
    n = len(b)
    x = np.random.rand(n)
    k = np.random.rand(n)
    
    for _ in range(1000):
        for i in range(n):
            sum_ = 0
            for j in range(n):   
                if i != j:
                    sum_ += A[i, j] * k[j]
            
            x[i] = (b[i] - sum_)/A[i, i]
        
        if np.linalg.norm(x-k)/np.linalg.norm(x) < 1e-9:
            break
        else:
            k = x.copy()
            
    return k

## Gauss-Seidel
![seidel](images/seidel.jpg)

In [14]:
def seidel(A, b):
    # https://www3.nd.edu/~zxu2/acms40390F12/Lec-7.3.pdf
    
    n = len(b)
    x = np.random.rand(n)
    k = np.random.rand(n)
    
    for _ in range(1000):
        for i in range(n):
            sum_ = 0
            for j in range(n):   
                if i != j:
                    sum_ += A[i, j] * x[j]
            
            x[i] = (b[i] - sum_)/A[i, i]
        
        if np.linalg.norm(x - k)/np.linalg.norm(x) < 1e-9:
            break
        else:
            k = x.copy()
            
    return x

## Testes

In [15]:
A = np.array([[4,1,1], [1,2,-1], [1,-1,3]], dtype=np.float64)
b = np.array([3,1,3/2], dtype=np.float64)

# A = np.array([[3,-2,1], [1,-3,2], [-1,2,4]], dtype=np.float64)
# b = np.array([3,1,3/2], dtype=np.float64)

In [16]:
np.linalg.solve(A,b)

array([0.5, 0.5, 0.5])

In [17]:
gauss = list(gaussian_elimination(A, b))
solve_upper(gauss[0], gauss[1])

[0.5, 0.5, 0.5]

In [18]:
solve_cholesky(A, b)

[0.5, 0.5, 0.5000000000000001]

In [19]:
jacobi(A,b)

array([0.5, 0.5, 0.5])

In [20]:
seidel(A,b)

array([0.5, 0.5, 0.5])